In [ ]:
# %pip install datasets
# %pip install numpy
# %pip install Pillow
# %pip install opencv-python
# %pip install json
# %pip install torch
# %pip install matplotlib

In [1]:
from datasets import load_dataset
import numpy as np
from PIL import Image
import cv2
import json

# import os
# import random
# import matplotlib.pyplot as plt
# import torch

KeyboardInterrupt: 

In [ ]:
# class DatasetReceipt(Dataset):
#     def __init__(self, hf_datasets=None, local_folder=None, local_format="csv"):
#         """
#         Parameters:
#         - hf_datasets: list berisi nama dataset dari Hugging Face (string)
#         - local_folder: path ke folder yang berisi dataset lokal
#         - local_format: format dataset lokal, default "csv", bisa "parquet" atau lainnya
#         """
#         self.data = []

#         # Load dataset dari Hugging Face
#         if hf_datasets:
#             for dataset_name in hf_datasets:
#                 dataset = load_dataset(dataset_name, split="train")  # Load train split
#                 for item in dataset:
#                     self.data.append({"text": item["text"], "label": item.get("label", -1)})  # Sesuaikan kolomnya

#         # Load dataset dari folder lokal
#         if local_folder:
#             for file in os.listdir(local_folder):
#                 file_path = os.path.join(local_folder, file)
#                 if file.endswith(".csv") and local_format == "csv":
#                     df = pd.read_csv(file_path)
#                 elif file.endswith(".parquet") and local_format == "parquet":
#                     df = pd.read_parquet(file_path)
#                 else:
#                     continue

#                 for _, row in df.iterrows():
#                     self.data.append({"text": row["text"], "label": row.get("label", -1)})

#     def __len__(self):
#         return len(self.data)

#     def __getitem__(self, idx):
#         return self.data[idx]


In [ ]:
# dataset_international = load_dataset("katanaml-org/invoices-donut-data-v1")
# sample_image_international = dataset_international['train'][0]['image']
# print(sample_image_international)

# plt.imshow(sample_image_international)
# plt.axis("off")
# plt.show()

# mean = np.array([0.485])
# std = np.array([0.229])

# def resize_image(image):
#     """ Convert PIL image to NumPy array and resize """
#     if isinstance(image, Image.Image):  # Jika image berupa PIL.Image, konversi ke NumPy array
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("resize_image: Input harus berupa NumPy array")

#     return cv2.resize(image, (600, 600))

# def grey_image(image):
#     """ Convert image to grayscale """
#     if isinstance(image, Image.Image):
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("grey_image: Input harus berupa NumPy array")

#     return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# def normalize_image(image):
#     """ Normalize image using mean and std """
#     if isinstance(image, Image.Image):
#         image = np.array(image)

#     if not isinstance(image, np.ndarray):
#         raise ValueError("normalize_image: Input harus berupa NumPy array")

#     image = image / 255.0
#     image = (image - mean) / std

#     return Image.fromarray((image * 255).astype(np.uint8))  # Kembalikan ke PIL image

# def preprocess(batch):
#     try:
#         batch['image'] = [resize_image(img) for img in batch['image']]
#         batch['image'] = [grey_image(img) for img in batch['image']]
#         # batch['image'] = [normalize_image(img) for img in batch['image']]

#         return batch
#     except Exception as e:
#         print(f"Error dalam preprocess: {e}")
#         return batch  # Supaya tidak crash

# dataset_international = dataset_international.map(preprocess, batched=True, num_proc=1)  # Gunakan num_proc=1 dulu

# sample_image_international_normalization = dataset_international['train'][0]['image']
# print(sample_image_international_normalization)

# plt.imshow(sample_image_international_normalization)
# plt.axis("off")
# plt.show()

In [ ]:
# dataset = load_dataset("naver-clova-ix/cord-v2", split="train")
# for sample in dataset:
#     print(sample["ground_truth"])

In [ ]:
class DatasetReceipt:
    def __init__(self, dataset_name="naver-clova-ix/cord-v2", split="train"):
        """
        Parameters:
        - dataset_name: Nama dataset dari Hugging Face (default: "naver-clova-ix/cord-v2").
        - split: Split dataset yang akan digunakan (default: "train").
        """
        # Load dataset dari Hugging Face
        self.dataset = load_dataset(dataset_name, split=split)

    def __len__(self):
        """Mengembalikan jumlah sampel dalam dataset."""
        return len(self.dataset)

    def __getitem__(self, idx):
        """
        Mengambil sampel dari dataset berdasarkan indeks.

        Parameters:
        - idx: Indeks sampel yang akan diambil.

        Returns:
        - Dict berisi gambar yang sudah dipreprocess dan label (ground truth).
        """
        sample = self.dataset[idx]
        image = sample["image"]

        # Preprocess the image (menggunakan NumPy dan OpenCV)
        image = self.preprocess_image(image)

        # Get ground_truth dan convert dari JSON string ke dict
        ground_truth_str = sample["ground_truth"]
        print(f"Sample {idx} - Ground Truth String: {ground_truth_str}")  # Debugging

        try:
            ground_truth = json.loads(ground_truth_str)  # Convert to dict
            print(f"Sample {idx} - Parsed Ground Truth: {ground_truth}")  # Debugging
        except json.JSONDecodeError as e:
            print(f"Sample {idx} - Error decoding JSON: {e}")
            ground_truth = {}  # Default jika parsing gagal

        return {"image": image, "label": ground_truth}

    def preprocess_image(self, image):
        if isinstance(image, Image.Image):  # Jika formatnya PIL, ubah ke NumPy
            image = np.array(image)
            # image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        # Resize ke (600, 600)
        image = cv2.resize(image, (600, 600))

        # Convert ke grayscale jika masih RGB
        if len(image.shape) == 3:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        return image  # Output dalam format NumPy


In [ ]:
# dataset = DatasetReceipt()
# sample = dataset[0]  # Get the first sample
# print("Sample Output:")
# print("Image Shape:", sample["image"].shape)
# print("Label:", sample["label"])
# Verify dataset
# for i in range(5):
#     sample = dataset[i]
#     print(f"Sample {i}:")
#     print("Image Shape:", sample["image"].shape)
#     print("Label:", sample["label"])
#     print("-" * 50)


Sample 0 - Ground Truth String: {"gt_parse": {"menu": [{"nm": "Nasi Campur Bali", "cnt": "1 x", "price": "75,000"}, {"nm": "Bbk Bengil Nasi", "cnt": "1 x", "price": "125,000"}, {"nm": "MilkShake Starwb", "cnt": "1 x", "price": "37,000"}, {"nm": "Ice Lemon Tea", "cnt": "1 x", "price": "24,000"}, {"nm": "Nasi Ayam Dewata", "cnt": "1 x", "price": "70,000"}, {"nm": "Free Ice Tea", "cnt": "3 x", "price": "0"}, {"nm": "Organic Green Sa", "cnt": "1 x", "price": "65,000"}, {"nm": "Ice Tea", "cnt": "1 x", "price": "18,000"}, {"nm": "Ice Orange", "cnt": "1 x", "price": "29,000"}, {"nm": "Ayam Suir Bali", "cnt": "1 x", "price": "85,000"}, {"nm": "Tahu Goreng", "cnt": "2 x", "price": "36,000"}, {"nm": "Tempe Goreng", "cnt": "2 x", "price": "36,000"}, {"nm": "Tahu Telor Asin", "cnt": "1 x", "price": "40,000."}, {"nm": "Nasi Goreng Samb", "cnt": "1 x", "price": "70,000"}, {"nm": "Bbk Panggang Sam", "cnt": "3 x", "price": "366,000"}, {"nm": "Ayam Sambal Hija", "cnt": "1 x", "price": "92,000"}, {"nm":